In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd

import re
import json
import requests
from tqdm import tqdm
from datetime import date

In [2]:
driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver")

C:\Users\Admin\AppData\Local\Temp\ipykernel_12320\2531624283.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver")


In [3]:
data = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220819/LaRochePosay_Infomation.csv')

In [4]:
total_link = data.Link.dropna()

In [5]:
"""
    Vì Category của Sendo nó hiển thị như thế này
        [Sendo.vn/Sức Khỏe & Làm Đẹp/Trị liệu chuyên sâu da mặt/Điều trị, chăm sóc da mụn]
    Nên hàm dưới đây sẽ đưa nó về theo format như dưới đấy
        [Sendo.vn - Sức Khỏe & Làm Đẹp - Trị liệu chuyên sâu da mặt - Điều trị, chăm sóc da mụn]
"""


def get_category(driver):
    list_category= driver.find_elements(by=By.XPATH, value='//span[@class="undefined d7e-4a1d04 d7e-b61d5e"]')
    category_detail = ''

    for sub_category in list_category:
        if(category_detail == ''):
            category_detail = category_detail +  sub_category.text
        else:
            category_detail = category_detail + " - " + sub_category.text
    return category_detail

In [8]:
Product_ID = [] # Product_ID của sản phẩm
Name = [] # Tên của sản phẩm
Min_Sell_Price = [] # Giá thấp nhất của sản phẩm - sẽ cập nhật lại theo code của Henry hoặc Will
Last_Crawl_Sell_Price = [] # Giá của sản phẩm hiện tại (sau khi crawl)
Location = []  # Nơi bán sản phẩm
Product_Link = [] # Link sản phẩm
Min_Price_Crawl_Date = [] # Ngày crawl được giá nhỏ nhất
Last_Crawl_Date = [] # Ngày crawl sản phẩm
Regular_Price = [] # Giá mặc định khi chưa áp dụng khuyến mãi
Rating_Score = [] # Điểm trung bình đánh giá của sản phẩm
Rating_Count = [] # Số lần đánh giá sản phẩm
Description = [] # Mô tả, chú thích sản phẩm
Category = [] # Danh mục của sản phẩm - VD: Sendo.vn - Đồ điện gia dụng - Nồi điện, nồi cơm điện - Nồi cơm điện
Sold = [] # Số sản phẩm đã bán được
Shop_ID = [] # Shop ID

In [ ]:
"""
    biến n để check số lượng sản phẩm lấy được
"""

n = 1

for _link in total_link:
    driver.get(_link)
    sleep(1)
    """
        Create API Link
    """
    _link_api_0 = str(_link).split("sendo.vn/")
    _link_api = _link_api_0[1].split(".html?")
    _link_api = 'https://detail-api.sendo.vn/full/' + _link_api[0]
    data = requests.get(
        _link_api
    ).json()
    sleep(1)

    _Product_ID = data['data']['id']
    _Name = data['data']['name']
    # Min_Sell_Price
    _Last_Crawl_Sell_Price = data['data']['final_price']

    try:
        location_average_item = driver.find_element(by=By.XPATH, value='//span[@class="d7e-ed528f d7e-fde242 d7e-7ef9b0 d7e-b61d5e d7e-a58302"]').text
        if("|" in location_average_item):
            _Location = location_average_item.split('|')[0]
        else:
            _Location = location_average_item
    except:
        _Location = ''


    _Product_Link = _link
    # _Min_Price_Crawl_Date
    _Last_Crawl_Date = date.today()
    _Regular_Price = data['data']['price']

    try:
        _Rating_Score = driver.find_element(by=By.XPATH, value='//span[@class="d7e-ed528f d7e-fde242 d7e-7ef9b0 d7e-b61d5e d7e-a58302"]').text.split(' ')[-2]
    except:
        _Rating_Score = ''
    try:
        _Rating_Count = re.findall('\d+', driver.find_element(by=By.XPATH, value='//span[@class="_314-995483 d7e-d87aa1 d7e-b61d5e d7e-65390f"]').text)[0]
    except:
        _Rating_Count = '0'

    try:
        _Description = driver.find_element(by=By.XPATH, value='//div[@class="d7e-9d72f3 d7e-e6ae34 e9f-847905"]').text
    except:
        _Description = ''
    _Category = get_category(driver)
    try:
        _Sold = re.findall('\d+',(driver.find_element(by=By.XPATH, value='//span[@class="_314-13ea6b d7e-d87aa1 d7e-b61d5e d7e-a58302"]').text))[0]
    except:
        _Sold = '0'
    _Shop_id = data['data']['shop_info']['shop_id']

    Product_ID.append(_Product_ID)
    Name.append(_Name)
    # Min_Sell_Price = [] # Gia Min cua san pham
    Last_Crawl_Sell_Price.append(_Last_Crawl_Sell_Price)
    Location.append(_Location)
    Product_Link.append(_Product_Link)
    # Min_Price_Crawl_Date = [] # Ngay crawl duoc gia nho nhat
    Last_Crawl_Date.append(_Last_Crawl_Date)
    Regular_Price.append(_Regular_Price)
    Rating_Score.append(_Rating_Score)
    Rating_Count.append(_Rating_Count)
    Description.append(_Description)
    Category.append(_Category)
    Sold.append(_Sold)
    Shop_ID.append(_Shop_id)

    print('Da lay duoc san pham thu ' + str(n))
    n += 1
    #  print(check_Location)
    print(_Location)
    if(n == 20):
        break

In [10]:
df_output = pd.DataFrame(list(zip(Product_ID, Name, Last_Crawl_Sell_Price, Location
                                    ,Product_Link, Last_Crawl_Date, Regular_Price, Rating_Score, Rating_Count,
                                    Description, Category, Sold, Shop_ID)),
               columns =['Product_ID', 'Name', 'Last_Crawl_Sell_Price', 'Location',
               'Product_Link', 'Last_Crawl_Date', 'Regular_Price', 'Rating_Score', 'Rating_Count',
                                    'Description', 'Category', 'Sold', 'Shop_ID'])

In [11]:
df_output.to_excel('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220819/Test_LA.xlsx')